<img src="../../Master/NotebookAddons/HydroSARbanner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="6"> <b>HYDRO30 Product: Flood Mapping from Single Sentinel-1 SAR Images</b><img style="padding: 7px" src="../../Master/NotebookAddons/UAFLogo_A_647.png" width="170" align="right"/></font>

<br>
<font size="4"> <b> Franz J Meyer, University of Alaska Fairbanks</b> <br>
</font>

<font size="3">This approach is based on a methodology developed by the German Aerospace Center and published in <cite><a href="https://www.tandfonline.com/doi/full/10.1080/01431161.2016.1192304"><i>Sentinel-1-based flood mapping: a fully automated processing chain</i></a> by Twele et al.</cite>. It is based on single-image Sentinel-1 SAR data and applies a dynamic thresholding method followed by fuzzy-logic-based post processing procedure. Modifications were made to the original approach to improve robustness and correct issues where the approach tended to fail.
</font>
<hr>

In [ ]:

%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
kernel.execute(command)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/hydrosar':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "hydrosar" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "hydrosar" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "hydrosar" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

# General Methodology and Workflow

<font face="Calibri" size="3">The workflow of the Sentinel-1-based processing chain, as outlined in the figure below, is composed of the following main elements: 
- <b>Step 1:</b> Automatic fetching of newly incoming SAR data over our area of interest and automatic processing to radiometric terrain corrected (RTC) products using the <a href="https://hyp3.asf.alaska.edu/">ASF HyP3 service</a>.  
- <b>Step 2:</b> initial classification using automatic thresholding, resulting in an initial flood mapping product (<b>Step 3</b>).
- <b>Step 4:</b> fuzzy-logic-based classification refinement, 
- <b>Step 5:</b> final classification including auxiliary data, and 
- <b>Step 6:</b> dissemination of the results. 


<img style="padding: 7px" src="../../Master/NotebookAddons/Watermappingworkflow2.jpg" width="80%" align="center"/>
</font>


# Flood Mapping Procedure

## Loading Python Libraries

In [ ]:
# Loading a number of Phython Libraries that are used in this Notebook
from pathlib import Path
import json
from typing import Tuple

from osgeo import osr
from osgeo import gdal
import numpy as np

import ipywidgets as ui

import rasterio
import pyproj
import skfuzzy
from tqdm.auto import tqdm
import astropy
import astropy.convolution


import warnings #Suppress warnings on occasion

import asf_notebook as asfn
asfn.jupytertheme_matplotlib_format()
%matplotlib notebook

## Helper Scripts To Set Up the Flood Mapping Approach

<font face="Calibri" size="3"><b>Write a function to pad an image, so it may be split into tiles with consistent dimensions</b></font>

In [ ]:
# Function to Pad the image
def pad_image(image: np.ndarray, to: int) -> np.ndarray:
    height, width = image.shape

    n_rows, n_cols = get_tile_row_col_count(height, width, to)
    new_height = n_rows * to
    new_width = n_cols * to

    padded = np.zeros((new_height, new_width))
    padded[:image.shape[0], :image.shape[1]] = image
    return padded

<font face="Calibri" size="3"><b>Write a function to tile an image</b></font>

In [ ]:
# Function for Image Tiling
def tile_image(image: np.ndarray, width, height) -> np.ndarray:
    _nrows, _ncols = image.shape
    _strides = image.strides

    nrows, _m = divmod(_nrows, height)
    ncols, _n = divmod(_ncols, width)

    assert _m == 0, "Image must be evenly tileable. Please pad it first"
    assert _n == 0, "Image must be evenly tileable. Please pad it first"

    return np.lib.stride_tricks.as_strided(
        np.ravel(image),
        shape=(nrows, ncols, height, width),
        strides=(height * _strides[0], width * _strides[1], *_strides),
        writeable=False
    ).reshape(nrows * ncols, height, width)


<font face="Calibri" size="3"><b>Write a function for multi-class Expectation Maximization Thresholding.</b></font>

In [ ]:
# Function for Threshold Calculation using an Expectation Maximization Approach
def EMSeg_opt(image, number_of_classes):
    image_copy = image.copy()
    image_copy2 = np.ma.filled(image.astype(float), np.nan) # needed for valid posterior_lookup keys
    image = image.flatten()
    minimum = np.amin(image)
    image = image - minimum + 1
    maximum = np.amax(image)

    size = image.size
    histogram = make_histogram(image)
    nonzero_indices = np.nonzero(histogram)[0]
    histogram = histogram[nonzero_indices]
    histogram = histogram.flatten()
    class_means = (
            (np.arange(number_of_classes) + 1) * maximum /
            (number_of_classes + 1)
    )
    class_variances = np.ones((number_of_classes)) * maximum
    class_proportions = np.ones((number_of_classes)) * 1 / number_of_classes
    sml = np.mean(np.diff(nonzero_indices)) / 1000
    iteration = 0
    while(True):
        class_likelihood = make_distribution(
            class_means, class_variances, class_proportions, nonzero_indices
        )
        sum_likelihood = np.sum(class_likelihood, 1) + np.finfo(
                class_likelihood[0][0]).eps
        log_likelihood = np.sum(histogram * np.log(sum_likelihood))
        for j in range(0, number_of_classes):
            class_posterior_probability = (
                histogram * class_likelihood[:,j] / sum_likelihood
            )
            class_proportions[j] = np.sum(class_posterior_probability)
            class_means[j] = (
                np.sum(nonzero_indices * class_posterior_probability)
                    / class_proportions[j]
            )
            vr = (nonzero_indices - class_means[j])
            class_variances[j] = (
                np.sum(vr *vr * class_posterior_probability)
                    / class_proportions[j] +sml
            )
            del class_posterior_probability, vr
        class_proportions = class_proportions + 1e-3
        class_proportions = class_proportions / np.sum(class_proportions)
        class_likelihood = make_distribution(
            class_means, class_variances, class_proportions, nonzero_indices
        )
        sum_likelihood = np.sum(class_likelihood, 1) + np.finfo(
                class_likelihood[0,0]).eps
        del class_likelihood
        new_log_likelihood = np.sum(histogram * np.log(sum_likelihood))
        del sum_likelihood
        if((new_log_likelihood - log_likelihood) < 0.000001):
            break
        iteration = iteration + 1
    del log_likelihood, new_log_likelihood
    class_means = class_means + minimum - 1
    s = image_copy.shape
    posterior = np.zeros((s[0], s[1], number_of_classes))
    posterior_lookup = dict()
    for i in range(0, s[0]):
        for j in range(0, s[1]):
            pixel_val = image_copy2[i,j] 
            if pixel_val in posterior_lookup:
                for n in range(0, number_of_classes): 
                    posterior[i,j,n] = posterior_lookup[pixel_val][n]
            else:
                posterior_lookup.update({pixel_val: [0]*number_of_classes})
                for n in range(0, number_of_classes): 
                    x = make_distribution(
                        class_means[n], class_variances[n], class_proportions[n],
                        image_copy[i,j]
                    )
                    posterior[i,j,n] = x * class_proportions[n]
                    posterior_lookup[pixel_val][n] = posterior[i,j,n]
    return posterior, class_means, class_variances, class_proportions

def make_histogram(image):
    image = image.flatten()
    indices = np.nonzero(np.isnan(image))
    image[indices] = 0
    indices = np.nonzero(np.isinf(image))
    image[indices] = 0
    del indices
    size = image.size
    maximum = int(np.ceil(np.amax(image)) + 1)
    #maximum = (np.ceil(np.amax(image)) + 1)
    histogram = np.zeros((1, maximum))
    for i in range(0,size):
        #floor_value = int(np.floor(image[i]))
        floor_value = np.floor(image[i]).astype(np.uint8)
        #floor_value = (np.floor(image[i]))
        if floor_value > 0 and floor_value < maximum - 1:
            temp1 = image[i] - floor_value
            temp2 = 1 - temp1
            histogram[0,floor_value] = histogram[0,floor_value] + temp1
            histogram[0,floor_value - 1] = histogram[0,floor_value - 1] + temp2
    histogram = np.convolve(histogram[0], [1,2,3,2,1])
    histogram = histogram[2:(histogram.size - 3)]
    histogram = histogram / np.sum(histogram)
    return histogram

def make_distribution(m, v, g, x):
    x = x.flatten()
    m = m.flatten()
    v = v.flatten()
    g = g.flatten()
    y = np.zeros((len(x), m.shape[0]))
    for i in range(0,m.shape[0]):
        d = x - m[i]
        amp = g[i] / np.sqrt(2*np.pi*v[i])
        y[:,i] = amp * np.exp(-0.5 * (d * d) / v[i])
    return y

<font face="Calibri" size="3"><b>Write a function to calculate the number of rows and columns of tiles needed to tile an image to a given size</b></font>

In [ ]:
# Function to Calculate Number of Image Tiles
def get_tile_row_col_count(height: int, width: int, tile_size: int) -> Tuple[int, int]:
    return int(np.ceil(height / tile_size)), int(np.ceil(width / tile_size))

<font face="Calibri" size="3"><b>Write a function to extract the tiff dates from a wildcard path:</b> </font>

In [ ]:
# Function to Extract Image Acquisition Date Information
def get_dates(paths):
    dates = []
    pths = paths.parent.rglob(paths.name)
    for p in pths:
        date = str(p).split('/')[-1].split("_")[3].split("T")[0]
        dates.append(date)
    dates.sort()
    return dates

<font face="Calibri" size="3"><b>Write a function to save a mask</b></font>

In [ ]:
# Some Additional Python Utilities
def write_mask_to_file(mask: np.ndarray, file_name: str, projection: str, geo_transform: str) -> None:
    (width, height) = mask.shape
    out_image = gdal.GetDriverByName('GTiff').Create(
        file_name, height, width, bands=1
    )
    out_image.SetProjection(projection)
    out_image.SetGeoTransform(geo_transform)
    out_image.GetRasterBand(1).WriteArray(mask)
    out_image.GetRasterBand(1).SetNoDataValue(0)
    out_image.FlushCache()

def gdal_write(ary, geoTransform, fileformat="GTiff", filename='jupyter_rocks.tif', format=gdal.GDT_Float64, nodata=None, srs_proj4='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'):
    '''gdal_write(ary, geoTransform, format="GTiff", filename='jupyter_rocks.tif', format=gdal.GDT_Float64 nodata=None, srs_proj4='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
    ary: 2D array.
    geoTransform: [top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution]
    format: "GTiff"     
    '''           
    if ary.ndim ==2:
      Ny, Nx = ary.shape
      Nb = 1;
    elif ary.ndim==3:
      Ny,Nx,Nb=ary.shape
    else: 
      print("Input array has to be 2D or 3D.")
      return None
    
    driver = gdal.GetDriverByName(fileformat)
    ds = driver.Create(filename, Nx, Ny, Nb, gdal.GDT_Float64)

    #ds.SetGeoTransform( ... ) # define GeoTransform tuple
    # top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
    ds.SetGeoTransform( geoTransform )    
    srs=osr.SpatialReference()
    srs.ImportFromProj4(srs_proj4)
    ds.SetProjection(srs.ExportToWkt() );
    if nodata is not None:
        ds.GetRasterBand(1).SetNoDataValue(0);
    if Nb==1:
      ds.GetRasterBand(1).WriteArray(ary)
    else:
      for b in range(Nb):
        ds.GetRasterBand(b+1).WriteArray(ary[:,:,b])
    ds = None
    #print("File written to: " + filename);

In [ ]:
# Function To Select Path to Folder Holding your HAND File
class PathSelector():

    def __init__(self,start_dir,select_file=True):
        self.file        = None 
        self.select_file = select_file
        self.cwd         = start_dir
        self.select      = ui.SelectMultiple(options=['init'],value=(),rows=10,description='') 
        self.accord      = ui.Accordion(children=[self.select]) 

        self.accord.selected_index = None # Start closed (showing path only)
        self.refresh(self.cwd)
        self.select.observe(self.on_update,'value')

    def on_update(self,change):
        if len(change['new']) > 0:
            self.refresh(change['new'][0])

    def refresh(self,item):
        path = (self.cwd/item).resolve()

        if path.is_file():
            if self.select_file:
                self.accord.set_title(0,str(path))  
                self.file = path
                self.accord.selected_index = None
            else:
                self.select.value = ()

        else: # if path is dir
            self.file = None 
            self.cwd  = path

            # Build list of files and dirs
            keys = ['[..]']; 
            for item in path.iterdir():
                if item == Path.cwd():
                    continue
                elif (path/item).is_dir():
                    keys.append(f'[{item.name}]'); 
                else:
                    keys.append(item.name); 

            # Sort and create list of output values
            keys.sort(key=str.lower)
            vals = []
            for k in keys:
                if k[0] == '[':
                    vals.append(k[1:-1]) # strip off brackets
                else:
                    vals.append(k)

            # Update widget
            self.accord.set_title(0,str(path))  
            self.select.options = list(zip(keys,vals)) 
            with self.select.hold_trait_notifications():
                self.select.value = ()
                
def get_proj4(filename):
    f=rasterio.open(filename)
    return pyproj.Proj(f.crs, preserve_units=True)  #used in pysheds
    
def gdal_read(filename, ndtype=np.float64):
    '''
    z=readData('/path/to/file')
    '''
    ds = gdal.Open(filename) 
    return np.array(ds.GetRasterBand(1).ReadAsArray()).astype(ndtype);
def gdal_get_geotransform(filename):
    '''
    [top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution]=gdal_get_geotransform('/path/to/file')
    '''
    #http://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
    ds = gdal.Open(filename)
    return ds.GetGeoTransform()

def fill_nan(arr):
    """
    filled_arr=fill_nan(arr)
    Fills Not-a-number values in arr using astropy. 
    """    
    kernel = astropy.convolution.Gaussian2DKernel(x_stddev=3) #kernel x_size=8*stddev
    arr_type=arr.dtype          
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        while np.any(np.isnan(arr)):
            arr = astropy.convolution.interpolate_replace_nans(arr.astype(float), kernel, convolve=astropy.convolution.convolve)
    return arr.astype(arr_type) 

def get_tiff_paths(paths: str) -> list:
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

## Find SAR Data Sets to Process

<font face="Calibri" size="3"><b>Enter the path to the data stack</b></font>

In [ ]:
# Define Path to Folder Containing Your SAR Images
while True:
    print("Enter the absolute path to the directory holding your tiffs.")
    tiff_dir = Path(input())
    paths = tiff_dir/"*_V*.tif*"
    if tiff_dir.exists():
        tiff_paths = get_tiff_paths(paths)
        if len(tiff_paths) < 1:
            print(f"{tiff_dir} exists but contains no tifs.")
            print("You will not be able to proceed until tifs are prepared.")
        break
    else:
        print(f"\n{tiff_dir} does not exist.")
        continue

<font face="Calibri" size="3"><b>Move into the parent directory of the directory containing the data and create a directory in which to store the water masks</b></font>

In [ ]:
# Move into Data Directory for Processing

mask_directory = tiff_dir.parent/'Water_Masks'

if not mask_directory.exists():
    mask_directory.mkdir()

<font face="Calibri" size="3"><b>Write a function to create a dictionary containing lists of each vv/vh pair</b></font>

In [ ]:
# Define Function for Grouping VV / VH 
def group_polarizations(tiff_paths: list) -> dict:
    pths = {}
    for tiff in tiff_paths:
        product_name = tiff.split('.')[0][:-2]
        if product_name in pths:
            pths[product_name].append(tiff)
        else:
            pths.update({product_name: [tiff]})
            pths[product_name].sort()
    return pths

<font face="Calibri" size="3"><b>Write a function to confirm the presence of both VV and VH images in all image sets</b></font>

In [ ]:
# Define Function to Confirm that every Image Date has Both VV and VH Polarization
def confirm_dual_polarizations(paths: dict) -> bool:
    for p in paths:
        if len(paths[p]) == 2:
            if ('vv' not in paths[p][1] and 'VV' not in paths[p][1]) or \
            ('vh' not in paths[p][0] and 'VH' not in paths[p][0]):
                return False
    return True   

<font face="Calibri" size="3"><b>Create a dictionary of VV/VH pairs and check it for completeness</b></font>

In [ ]:
# Group VV and VH Images by Date and make sure Every Date has both Polarizations 
grouped_pths = group_polarizations(tiff_paths)
if not confirm_dual_polarizations(grouped_pths):
    print("ERROR: AI_Water requires both VV and VH polarizations.")
else:
    print("Confirmed presence of VV and VH polarities for each product.")
    
#print(grouped_pths) #uncomment to print VV/VH path pairs

## Identify HAND Layer to be used in Flood Mapping

<font face="Calibri" size="3"><font color='rgba(200,0,0,0.2)'><b>Note:</b></font> Please pick a HAND layer that covers your entire area of interest (or more).</font>

In [ ]:
# Pick the HAND File you want to use
print("Choose your HAND layer:")
f = PathSelector(Path.cwd())
display(f.accord)

In [ ]:
# Read HAND Data and Projection Information
HAND_file=f.accord.get_title(0)
info = (gdal.Info(HAND_file, options = ['-json']))
info = json.dumps(info)
info = (json.loads(info))['coordinateSystem']['wkt']
Hzone = info.split('ID')[-1].split(',')[1][0:-2]
Hproj_type = info.split('ID')[-1].split('"')[1]

## Do Initial Flood Mapping using Adaptive Dynamic Thresholding

<font face="Calibri" size="4"><b>A bit of background on the approach:</b></font>

<font face="Calibri" size="3"><u><b>This is what's implemented so far to arrive at the initial flood mapping product </b> (approach modified from Twele et al., (2016))</u>: An automatic tile-based thresholding procedure (Martinis, Twele, and Voigt 2009) is used to generate an initial land/water classification. The selection of tiles is performed on a bilevel quadtree structure with parent level $L^+$ and child level $L^−$:
    
1. Firstly, the entire data are separated into quadratic non-overlapping parent tiles on level $L^+$ with a size of $200 \times 200$ pixels. Each parent object is further represented by four quadratic child objects on a second level $L^−$. The tile selection process is based on statistical hierarchical relations between parent and child objects. <font color='rgba(200,0,0,0.2)'><b>[Note: I modified this step slightly. Instead of starting with a $200 \times 200$ pixels tile, I am using $100 \times 100$ pixel tiles instead. This seems to be be more robust. This could be modified in the future</b></font>
2. A number of parent tiles is automatically selected which offer the highest (>95% quantile) standard deviation on $L^+$ of the mean backscatter values of the respective child objects on $L^−$. This criterion serves as a measure of the degree of variation within the data and can therefore be used as an indicator of the probability that the tiles are characterized by spatial inhomogeneity and contain more than one semantic class. <font color='rgba(200,0,0,0.2)'><b>[Note: I modified this step slightly. Instead of the standard deviation $\sigma$, I am using the coefficient of variation $\frac{\sigma}{\mu}$ here. This seems to be more stable.]</b></font> The selected parent objects should also have a mean individual backscatter value lower than the mean of all parent tiles on $L^+$. This ensures that tiles lying on the boundary between water and no water areas are selected. In case that no tiles fulfil these criteria, the tile size on $L^+$ and $L^−$ is halved and the quantile for the tile selection is reduced to 90% to guarantee a successful tile selection also in data with a relatively low extent of water surfaces or with smaller dispersed water bodies. 
3. <b>NEW - Integration of HAND</b>: To improve the robustness of the automatic threshold derivation it should be considered to restrict the tile selection in Step (3) to only pixels situated in flood-prone regions defined by a HAND-based binary Exclustion Mask (HAND-EM). To create HAND-EM, a threshold is applied to HAND to identify non-flood prone areas. A threshold value of $\geq 15m$ is proposed. The HAND-EM further is shrunk by one pixel using an 8-neighbour function to account for potential geometric inaccuracies between the exclude layer and SAR data. Tiles are only considered in case less than 20% of its data pixels are excluded by HAND-EM. <font color='rgba(200,0,0,0.2)'><b>This step is now included!</b></font>
4. Out of the number of the initially selected tiles, a limited number of N parent tiles are finally chosen for threshold computation. This selection is accomplished by ranking the parent tiles according to the standard deviation of the mean backscatter values of the respective child objects. Tiles with the highest values are chosen for $N$. Extensive testing yielded that $N = 5$ is a sufficient number of parent tiles for threshold computation. 
5. The parametric Kittler and Illingworth minimum error thresholding approach (Kittler and Illingworth 1986) is then employed to derive local threshold values using a cost function which is based on the statistical parameterization of the sub-histograms of all selected tiles as bi-modal Gaussian mixture distributions. In order to derive a global (i.e. scenebased) threshold, the locally derived thresholds are combined by computing their arithmetic mean. <font color='rgba(200,0,0,0.2)'><b>[Note: I replaced Kittler Illingworth with a multi-mode Expectation Maximization approach to be able to cope with regions that have three classes (urban and bright slopes; average brightness farm lands; water). This happens a lot in the midwest and Kittler Illingworth didn't perform well.]</b></font>
6. Using the dynamically calculated threshold, both the VV and VH scenes are thresholded for water detection
7. The detected water maps are combined to arrive at an intial water mask (<b>Step 3</b>) that is further refined in post processing (Section 2.6).


<font face="Calibri" size="4"><b>Now Let's do the Work:</b></font>

In [ ]:
print(type(grouped_pths))

for pair in grouped_pths:
    print(pair)
    print(type(pair))

In [ ]:
from os import system

# Perform Inital Surface Water Mask Creation

# define some variables you might want to change
precentile = 0.95        # Standard deviation percentile threshold for pivotal tile selection
tilesize = 100           # Setting tile size to use in thresholding algorithm
tilesize2 = 50
Hpick = 0.8              # Threshold for required fraction of valid HAND-EM pixels per tile
vv_corr = -17.0          # VV threshold to use if threshold calculation did not succeed
vh_corr = -24.0          # VH threshold to use if threshold calculation did not succeed
Hthresh = 15

# Now do adaptive threshold selection
vv_thresholds = np.array([])
vh_thresholds = np.array([])
floodarea = np.array([])
vh_thresholds_corr = np.array([])
vv_thresholds_corr = np.array([])

for pair in grouped_pths:
    for tiff in grouped_pths[pair]:
        
        print(tiff, ' has type: ', type(tiff))
        
        resampled_dem_path=f'{tiff_dir}/resamp_dem.tif'
        f = gdal.Open(tiff)
        img_array = f.ReadAsArray()
        original_shape = img_array.shape
        img_array = 0     # free up RAM
        
        info1 = (gdal.Info(tiff, options = ['-json']))
        info1 = json.dumps(info1)
        ul = (json.loads(info1))['cornerCoordinates']['upperLeft']
        lr = (json.loads(info1))['cornerCoordinates']['lowerRight']
        coordsys = (json.loads(info1))['coordinateSystem']['wkt']
        Szone = coordsys.split('ID')[-1].split(',')[1][0:-2]
        Sproj_type = coordsys.split('ID')[-1].split('"')[1]

        west= ul[0]
        east= lr[0]
        south= lr[1]
        north= ul[1]
        
        print('----------------------------------')
        print('Extracting Relevant Subset from HAND Layer')
        print('----------------------------------')
        
        cmd_resamp=f"gdalwarp -overwrite -s_srs {Hproj_type}:"\
        f"{Hzone} -t_srs EPSG:{Szone} -te {west} {south} {east} {north} -ts {original_shape[1]} {original_shape[0]} -r lanczos {HAND_file} {resampled_dem_path}"
        #print(cmd_resamp)
        system(cmd_resamp)
        
        g = gdal.Open(resampled_dem_path)
        hand = g.ReadAsArray()
        Hmask = hand < Hthresh
        handem = np.zeros_like(hand)
        sel = np.ones_like(hand)
        handem[Hmask] = sel[Hmask]
        hand = 0
        
        # Tile up HAND-EM data
        handem_p = pad_image(handem, tilesize)
        hand_tiles = tile_image(handem_p,width=tilesize,height=tilesize)
        Hsum = np.sum(hand_tiles, axis=(1,2))
        Hpercent = Hsum/(tilesize*tilesize)
        
        n_rows, n_cols = get_tile_row_col_count(*original_shape, tile_size=tilesize)
        print('----------------------------------')
        print(f'Flood Mapping on Image: {tiff}')
        print('----------------------------------')
        if 'vv' in tiff or 'VV' in tiff:
            vv_array = pad_image(f.ReadAsArray(), tilesize)
            invalid_pixels = np.nonzero(vv_array == 0.0)
            vv_tiles = tile_image(vv_array,width=tilesize,height=tilesize)
            a = np.shape(vv_tiles)
            vv_std = np.zeros(a[0])
            vvt_masked = np.ma.masked_where(vv_tiles<=0, vv_tiles)
            vv_picktiles = np.zeros_like(vv_tiles)
            for k in range(a[0]):
                vv_subtiles = tile_image(vvt_masked[k,:,:],width=tilesize2,height=tilesize2)
                vvst_mean = np.ma.mean(vv_subtiles, axis=(1,2))
                vvst_std = np.ma.std(vvst_mean)
                vv_std[k] = np.ma.std(vvst_mean) 
            
            # find tiles with largest standard deviations
            vv_mean = np.ma.median(vvt_masked, axis=(1,2))
            x_vv = np.sort(vv_std/vv_mean)
            y_vv = np.arange(1, x_vv.size+1) / x_vv.size
            
            percentile2 = precentile
            noconverge_vv = 0
            sort_index = 0
            while np.size(sort_index) < 5 and noconverge_vv == 0: 
                threshold_index_vv = np.ma.min(np.where(y_vv>percentile2))
                threshold_vv = x_vv[threshold_index_vv]
                #sd_select_vv = np.nonzero(vv_std/vv_mean>threshold_vv)
                s_select_vv = np.nonzero(vv_std/vv_mean>threshold_vv) 
                h_select_vv = np.nonzero(Hpercent > Hpick)               # Includes HAND-EM in selection
                sd_select_vv = np.intersect1d(s_select_vv, h_select_vv)
            
                # find tiles with mean values lower than the average mean
                omean_vv = np.ma.median(vv_mean[h_select_vv])
                mean_select_vv = np.nonzero(vv_mean<omean_vv)
            
                # Intersect tiles with large std with tiles that have small means
                msdselect_vv = np.intersect1d(sd_select_vv, mean_select_vv)
                sort_index = np.flipud(np.argsort(vv_std[msdselect_vv]))
                percentile2 = percentile2 - 0.01
                if percentile2 < 0.5:
                    noconverge_vv = 1
                    sort_index = threshold_index_vv
                    m_thresh_vv = (vv_corr/10.0+30)
                    print("Tile Selection Did Not Converge - Going with Default Threshold")
            
            if noconverge_vv < 1:
                finalselect_vv = sort_index[0:5]
            
            temp = np.ma.masked_where(vv_array<=0, vv_array)
            np.seterr(divide='ignore')
            np.seterr(invalid='ignore')
            dbvv = np.log10(temp)+30
                
            if noconverge_vv < 1:
                # find local thresholds for 5 "best" tiles in the image
                l_thresh_vv = np.zeros(5)
                EMthresh_vv = np.zeros(5)
                scaling = 256/(np.mean(dbvv) + 3*np.std(dbvv))
                #scaling = 256/(np.mean(vv_array) + 3*np.std(vv_array))
                np.seterr(divide='ignore')
                np.seterr(invalid='ignore')
                dbtile = np.log10(vvt_masked)+30
                for k in range(5):
                    test = dbtile[msdselect_vh[finalselect_vh[k]]] * scaling
                    #test = vvt_masked[msdselect_vv[finalselect_vv[k]]] * scaling
                    A = np.around(test)
                    A = A.astype(int)
                    #t_thresh = Kittler(A)
                    [posterior, cm, cv, cp] = EMSeg_opt(A, 3)
                    sorti = np.argsort(cm)
                    cms = cm[sorti]
                    cvs = cv[sorti]
                    cps = cp[sorti]
                    xvec = np.arange(cms[0],cms[1],step=.05)
                    x1 = make_distribution(cms[0], cvs[0], cps[0], xvec)
                    x2 = make_distribution(cms[1], cvs[1], cps[1], xvec)
                    dx = np.abs(x1 - x2)
                    diff1 = posterior[:,:,0] - posterior[:,:,1]
                    t_ind = np.argmin(dx)
                    EMthresh_vv[k] = xvec[t_ind]/scaling
                
                    #l_thresh_vv[k] = t_thresh / scaling
                    #dbtile = np.log10(vvt_masked)+30
                
                    # Mark Tiles used for Threshold Estimation
                    vv_picktiles[msdselect_vh[finalselect_vh[k]],:,:]= np.ones_like(vv_tiles[msdselect_vh[finalselect_vh[k]],:,:])
            
            if noconverge_vv < 1:
                # Calculate best threshold for VV and VH as the mean of the 5 thresholds calculated in the previous section 
                #m_thresh_vv = np.median(l_thresh_vv)
                #print(EMthresh_vv-30)
                EMts = np.sort(EMthresh_vv)
                #m_thresh_vv = np.median(EMthresh_vv)
                m_thresh_vv = np.median(EMts[0:4])
            
            print("Best VV Flood Mapping Threshold [dB]: %.2f" % (10*(m_thresh_vv-30)))
            print(" ")
            
            # Derive flood mask using the best threshold
            if m_thresh_vv < (vv_corr/10.0+30):
                change_mag_mask_vv = np.ma.masked_where(dbvv<=0, dbvv) < m_thresh_vv
                vv_thresholds_corr = np.append(vv_thresholds_corr, 10.0*(m_thresh_vv-30))
                #change_mag_mask_vv = np.ma.masked_where(vv_array==0, vv_array) < m_thresh_vv
            else:
                change_mag_mask_vv = np.ma.masked_where(dbvv<=0, dbvv) < (vv_corr/10.0+30)
                vv_thresholds_corr = np.append(vv_thresholds_corr, vv_corr)
            
            # Create Binary masks showing flooded pixels as "1"s
            flood_vv = np.zeros_like(vv_array)
            sel = np.ones_like(vv_array)
            flood_vv[change_mag_mask_vv] = sel[change_mag_mask_vv]
            np.putmask(flood_vv,vv_array==0 , 0)
            
            # Export flood maps as GeoTIFFs
            filename, ext = Path(tiff).name.split('.')
            outfile = f"{mask_directory}/{filename}_water_mask.{ext}"
            write_mask_to_file(flood_vv, outfile, f.GetProjection(), f.GetGeoTransform())
            #vv_array = 0
            maskedarray = 0
            temp = 0
            dbvv = 0
            change_mag_mask_vv = 0
            Hmask = 0
            Hpercent = 0
            Hsum = 0
            hand_tiles = 0
            handem = 0
            handem_p = 0
            vv_tiles = 0
        
        else:            
            vh_array = pad_image(f.ReadAsArray(), tilesize)
            invalid_pixels = np.nonzero(vh_array == 0.0)
            vh_tiles = tile_image(vh_array,width=tilesize,height=tilesize)
            a = np.shape(vh_tiles)
            vh_std = np.zeros(a[0])
            vht_masked = np.ma.masked_where(vh_tiles<=0, vh_tiles)
            vh_picktiles = np.zeros_like(vh_tiles)
            for k in range(a[0]):
                vh_subtiles = tile_image(vht_masked[k,:,:],width=tilesize2,height=tilesize2)
                vhst_mean = np.ma.mean(vh_subtiles, axis=(1,2))
                vhst_std = np.ma.std(vhst_mean)
                vh_std[k] = np.ma.std(vhst_mean)
            
            # find tiles with largest standard deviations
            vh_mean = np.ma.median(vht_masked, axis=(1,2))
            x_vh = np.sort(vh_std/vh_mean)
            xm_vh = np.sort(vh_mean)
            #x_vh = np.sort(vh_std)            
            y_vh = np.arange(1, x_vh.size+1) / x_vh.size
            ym_vh = np.arange(1, xm_vh.size+1) / xm_vh.size
            
            percentile2 = precentile
            noconverge_vh = 0
            sort_index = 0
            while np.size(sort_index) < 5 and noconverge_vh == 0: 
                threshold_index_vh = np.ma.min(np.where(y_vh>percentile2))
                threshold_vh = x_vh[threshold_index_vh]
                #sd_select_vh = np.nonzero(vh_std/vh_mean>threshold_vh)
                s_select_vh = np.nonzero(vh_std/vh_mean>threshold_vh) 
                h_select_vh = np.nonzero(Hpercent > Hpick)               # Includes HAND-EM in selection
                sd_select_vh = np.intersect1d(s_select_vh, h_select_vh)
    
                # find tiles with mean values lower than the average mean 
                omean_vh = np.ma.median(vh_mean[h_select_vh])
                mean_select_vh = np.nonzero(vh_mean<omean_vh)
            
                # Intersect tiles with large std with tiles that have small means
                msdselect_vh = np.intersect1d(sd_select_vh, mean_select_vh)
                sort_index = np.flipud(np.argsort(vh_std[msdselect_vh]))
                percentile2 = percentile2 - 0.01
                if percentile2 < 0.5:
                    noconverge_vh = 1
                    sort_index = threshold_index_vh
                    m_thresh_vh = (vh_corr/10.0+30)
                    print("Tile Selection Did Not Converge - Going with Default Threshold")
            
            if noconverge_vh < 1:
                finalselect_vh = sort_index[0:5]
    
            temp = np.ma.masked_where(vh_array<=0, vh_array)
            np.seterr(divide='ignore')
            np.seterr(invalid='ignore')
            dbvh = np.log10(temp)+30
            
            if noconverge_vh < 1:
                # find local thresholds for 5 "best" tiles in the image
                l_thresh_vh = np.zeros(5)
                EMthresh_vh = np.zeros(5)
                scaling = 256/(np.mean(dbvh) + 3*np.std(dbvh))
                #scaling = 256/(np.mean(vh_array) + 3*np.std(vh_array))
                np.seterr(divide='ignore')
                np.seterr(invalid='ignore')
                dbtile = np.log10(vht_masked)+30
                for k in range(5):
                    test = dbtile[msdselect_vh[finalselect_vh[k]]] * scaling
                    #test = vht_masked[msdselect_vh[finalselect_vh[k]]] * scaling
                    A = np.around(test)
                    A = A.astype(int)
                    #t_thresh = Kittler(A)
                    [posterior, cm, cv, cp] = EMSeg_opt(A, 3)
                    sorti = np.argsort(cm)
                    cms = cm[sorti]
                    cvs = cv[sorti]
                    cps = cp[sorti]
                    xvec = np.arange(cms[0],cms[1],step=.05)
                    x1 = make_distribution(cms[0], cvs[0], cps[0], xvec)
                    x2 = make_distribution(cms[1], cvs[1], cps[1], xvec)
                    dx = np.abs(x1 - x2)
                    diff1 = posterior[:,:,0] - posterior[:,:,1]
                    t_ind = np.argmin(dx)
                    EMthresh_vh[k] = xvec[t_ind]/scaling

                    #l_thresh_vh[k] = t_thresh / scaling


                    # Mark Tiles used for Threshold Estimation
                    vh_picktiles[msdselect_vh[finalselect_vh[k]],:,:]= np.ones_like(vh_tiles[msdselect_vh[finalselect_vh[k]],:,:])
    
            if noconverge_vh < 1:
                # Calculate best threshold for VV and VH as the mean of the 5 thresholds calculated in the previous section 
                #m_thresh_vh = np.median(l_thresh_vh)
                #print(EMthresh_vh-30)
                EMts = np.sort(EMthresh_vh)
                #m_thresh_vh = np.median(EMthresh_vh)
                m_thresh_vh = np.median(EMts[0:4])
            
            print("Best VH Flood Mapping Threshold [dB]: %.2f" % (10*(m_thresh_vh-30)))
            print(" ")
    
            # Derive flood mask using the best threshold
            maskedarray = np.ma.masked_where(dbvh<=0, dbvh)
            
            #maskedarray = np.ma.masked_where(vh_array==0, vh_array)
            if m_thresh_vh < (vh_corr/10.0+30):
                change_mag_mask_vh = maskedarray < m_thresh_vh
                vh_thresholds_corr = np.append(vh_thresholds_corr, 10.0*(m_thresh_vh-30)) 
                #change_mag_mask_vv = np.ma.masked_where(vv_array==0, vv_array) < m_thresh_vv
            else:
                change_mag_mask_vh = maskedarray < (vh_corr/10.0+30)
                vh_thresholds_corr = np.append(vh_thresholds_corr, vh_corr) 
            # change_mag_mask_vh = vh_array < m_thresh_vh
    
            # Create Binary masks showing flooded pixels as "1"s
            sel = np.ones_like(vh_array)
            flood_vh = np.zeros_like(vh_array)
            flood_vh[change_mag_mask_vh] = sel[change_mag_mask_vh]
            np.putmask(flood_vh,vh_array==0 , 0)

            # Export flood maps as GeoTIFFs
            filename, ext = Path(tiff).name.split('.')
            outfile = f"{mask_directory}/{filename}_water_mask.{ext}"
            write_mask_to_file(flood_vh, outfile, f.GetProjection(), f.GetGeoTransform())
            vh_array = 0
            maskedarray = 0
            temp = 0
            dbvh = 0
            change_mag_mask_vh = 0
            Hmask = 0
            Hpercent = 0
            Hsum = 0
            hand_tiles = 0
            handem = 0
            handem_p = 0
            vh_tiles = 0
            
        
    
    # Create Maps (Pickfiles) that show which tiles were used for adaptive threshold calculation
    if noconverge_vv < 1:
        vv_picks = vv_picktiles.reshape((n_rows, n_cols, tilesize, tilesize)) \
                    .swapaxes(1, 2) \
                    .reshape(n_rows * tilesize, n_cols * tilesize)  # yapf: disable
    if noconverge_vh < 1:
        vh_picks = vh_picktiles.reshape((n_rows, n_cols, tilesize, tilesize)) \
                    .swapaxes(1, 2) \
                    .reshape(n_rows * tilesize, n_cols * tilesize)  # yapf: disable
    vh_picktiles = 0
    vv_picktiles = 0
    
    # Write Pickfiles to GeoTIFFs
    #if noconverge_vv < 1:
    #    outfile = f"{mask_directory}/{filename[:-3]}_vv_pickfile.{ext}"
    #    write_mask_to_file(vv_picks, outfile, f.GetProjection(), f.GetGeoTransform())
    #if noconverge_vh < 1:
    #    outfile = f"{mask_directory}/{filename[:-3]}_vh_pickfile.{ext}"
    #    write_mask_to_file(vh_picks, outfile, f.GetProjection(), f.GetGeoTransform())

    # Combine VV and VH flood maps to produce a combined flood mapping product
    comb = flood_vh + flood_vv
    comb_mask = comb > 0
    flood_comb = np.zeros_like(vv_array)
    flood_comb[comb_mask] = sel[comb_mask]
    filename, ext = Path(tiff).name.split('.')
    outfile = f"{mask_directory}/{filename[:-3]}_water_mask_combined.{ext}"
    write_mask_to_file(flood_comb, outfile, f.GetProjection(), f.GetGeoTransform())
    
    # Create Information on Thresholds used as well as Flood extent information in km2
    vv_thresholds = np.append(vv_thresholds, 10.0*(m_thresh_vv-30))
    vh_thresholds = np.append(vh_thresholds, 10.0*(m_thresh_vh-30)) 
    floodarea = np.append(floodarea,(np.sum(flood_comb)*30**2./(1000**2)))
    flood_vh = 0
    flood_vv = 0

## Post Processing Steps to Clean up Initial Map

### Use Fuzzy Logic to Improve the Quality of The Water Mask

<font face="Calibri" size=3>This step applies fuzzy logic roles to remove spurious false detection and improve upon the initial flood mapping product created in Section 2.5. Four Fuzzy indicators for the presence of floods are used. These include:
<ol>
    <li>The radar cross section in a pixel relative to the determined detection threshold.</li>
    <li>The HAND elevation (surface elevation relative to the nearest dranage basin).</li>
    <li>The surface slope.</li>
    <li>The size of a detected flood feature.</li>
</ol>
Fuzzy membership functions are calculated for each of these four indicators using a Z-shaped activation function. Membership functions are combined using arithmetric averaging to form a final decision map. This map is then thresholded using a fuzzy threshold of 0.45. <br><br>

<b><u>Upper and lower thresholds for the fuzzy activation functions are calculated as follows:</u></b>
<ol>
    <li><b>RCS:</b> $\begin{Bmatrix} 
    x_{u,RCS} & = & \tau_g \\
    x_{l,RCS} & = & \mu_{\sigma^0(\tau_g)}
    \end{Bmatrix}$ 
    with $\sigma^0(\tau_g)$ = initial flood classification and flood mapping threshold $\tau_g$.<br><br></li>
    <li><b>HAND:</b> $\begin{Bmatrix} 
    x_{u,HAND} & = & \mu_{HAND(water)} + 3 \cdot \sigma_{HAND(water)} \\
    x_{l,HAND} & = & \mu_{HAND(water)}
    \end{Bmatrix}$. This is a departure from <cite><a href="https://www.sciencedirect.com/science/article/pii/S0924271614001981" target="_blank"><i>the paper</i></a> by Martinis et al. (2015)</cite>.<br><br></li>
    <li><b>Surface slope $\alpha$:</b> $\begin{Bmatrix} 
    x_{u,\alpha} & = & 0^{\circ} \\
    x_{l,\alpha} & = & 15^{\circ}
    \end{Bmatrix}$.<br><br></li>
    <li><b>Area $A$:</b>$\begin{Bmatrix} 
    x_{u,A} & = & 10px \\
    x_{l,A} & = & 3px
    \end{Bmatrix}$. These threshold values are different differently than in <cite><a href="https://www.sciencedirect.com/science/article/pii/S0924271614001981" target="_blank"><i>the paper</i></a> by Martinis et al. (2015)</cite>.</li>
</ol>
</font>

In [ ]:
# Perform Fuzzy Logic Post Processing of the Initial Flood Maps
import skimage.measure
import skimage.color
from skimage import morphology
from skimage import filters
import time
from IPython.display import clear_output
vvcount = 0
vhcount = 0

print('FUZZY LOGIC POST PROCESSING TO REFINE INITIAL SURFACE WATER EXTENT MAPS:')
print(' ')

for pair in grouped_pths:
    for tiff in grouped_pths[pair]:
        start = time.time()
        print('-----------------------------------------------------------------------------------')
        print(f'Image: {tiff}')
        resampled_dem_path=f'{tiff_dir}/resamp_dem.tif'
        f = gdal.Open(tiff)
        img_array = f.ReadAsArray()
        original_shape = img_array.shape
        img_array = 0     # free up RAM
        radar_array = pad_image(f.ReadAsArray(), tilesize)
        temp = np.ma.masked_where(radar_array==0, radar_array)
        np.seterr(divide='ignore')
        np.seterr(invalid='ignore')
        dbarray = np.log10(temp)+30
        del temp
        
        # ------------------------------------------------------------#
        # Loading Water Mask File                                     #
        #-------------------------------------------------------------#
        filename, ext = Path(tiff).name.split('.')
        waterfile = f"{mask_directory}/{filename}_water_mask.{ext}"
        h = gdal.Open(waterfile)
        maskimage = h.ReadAsArray()
    
        
        print('   - Extracting Relevant Subset from HAND Layer ...')

        info1 = (gdal.Info(tiff, options = ['-json']))
        info1 = json.dumps(info1)
        ul = (json.loads(info1))['cornerCoordinates']['upperLeft']
        lr = (json.loads(info1))['cornerCoordinates']['lowerRight']
        coordsys = (json.loads(info1))['coordinateSystem']['wkt']
        Szone = coordsys.split('ID')[-1].split(',')[1][0:-2]
        Sproj_type = coordsys.split('ID')[-1].split('"')[1]

        west= ul[0]
        east= lr[0]
        south= lr[1]
        north= ul[1]
        
        cmd_resamp=f"gdalwarp -overwrite -s_srs {Hproj_type}:"\
        f"{Hzone} -t_srs EPSG:{Szone} -te {west} {south} {east} {north} -ts {original_shape[1]} {original_shape[0]} -r lanczos {HAND_file} {resampled_dem_path}"
        #print(cmd_resamp)

        system(cmd_resamp)
        
        g = gdal.Open(resampled_dem_path)
        hand = pad_image(g.ReadAsArray(), tilesize)
        
        print('   - Interpolate NaNs in HAND Layer ...')
        
        hand_interp= fill_nan(hand)
        del hand
        
        print('   - Calculate DEM (HAND) Slope magnitude ...')
        
        vgrad = np.gradient(hand_interp)
        mag = np.sqrt(vgrad[0]**2 + vgrad[1]**2)
        geotransform = g.GetGeoTransform()
        res = geotransform[1]
        slope = np.arctan(mag/res)/np.pi*180

        print('   - Segment initial flood mask to calculate area of connected patches ...')
        
        # Here an attempt to perform a sequence of opening and closing steps to 
        # reduce the number of segments in the inital flood maps and speed up the next processing steps
        med = filters.median(maskimage, morphology.disk(2))
        selem = morphology.disk(3)
        closed = skimage.morphology.closing(med, selem)
        labeled_image = skimage.measure.label(closed, connectivity=2)
        label_areas = np.bincount(labeled_image.ravel())[1:]
        
        # Define upper and lower limit of the Z fuzzy activation function
        
        print('   - Now Calculate Fuzzy Weights ...')
        
        ## sigma zero upper and lower fuzzy threshold calculation
        maskedarray = np.ma.masked_where((maskimage==0) | (radar_array < 0), radar_array)
        db_lowerlimit = np.log10(np.ma.median(maskedarray))+30
        if 'vv' in tiff or 'VV' in tiff:
            db_upperlimit = (vv_thresholds_corr[vvcount]/10.0+30)
            vvcount = vvcount + 1
        else:
            db_upperlimit = (vh_thresholds_corr[vhcount]/10.0+30)
            vhcount = vhcount + 1
        
        ## HAND upper and lower fuzzy threshold calculation
        maskedarray = np.ma.masked_where(maskimage==0, hand_interp)
        ma2 = np.ma.masked_where(maskedarray > np.percentile(maskedarray, 90), maskedarray)
        hand_lowerlimit = np.ma.median(np.ma.masked_invalid(ma2))
        hand_upperlimit = hand_lowerlimit + (np.ma.std(np.ma.masked_invalid(ma2)) + 3.5)*np.ma.std(np.ma.masked_invalid(ma2))
        hand_upperlimit = hand_lowerlimit + 3.0*(np.ma.std(np.ma.masked_invalid(ma2)))

        # Create vector spanning all possible values for db, HAND, Slope, and Area values for the selected data set
        x_db = np.arange(np.min(dbarray), np.max(dbarray), 0.005)
        x_hand = np.arange(np.min(np.ma.masked_invalid(hand_interp)), np.max(np.ma.masked_invalid(hand_interp)), 0.1)
        x_slope = np.arange(np.min(np.ma.masked_invalid(slope)), np.max(np.ma.masked_invalid(slope)), 0.1)

        largestCC = labeled_image == np.argmax(np.bincount(labeled_image.flat, weights=maskimage.flat))
        x_area = np.arange(1, np.sum(largestCC)+10, 1)

        # Create activation functions for db, HAND, SLope and Area
        activation_db = skfuzzy.zmf(x_db,db_lowerlimit,db_upperlimit)
        activation_hand = skfuzzy.zmf(x_hand,hand_lowerlimit,hand_upperlimit)
        activation_slope = skfuzzy.zmf(x_slope,0,15)
        activation_area = 1-skfuzzy.zmf(x_area,3,10)
        
        # Calculate membership functions given your activation function rule
        db_membership = skfuzzy.interp_membership(x_db, activation_db, dbarray)
        print('      -- Calculating Radar Cross Section Membership ... [min thresholds: %6.2f' % (db_lowerlimit-30),'; max threshold: %6.2f]' % (db_upperlimit-30))     
        hand_membership = skfuzzy.interp_membership(x_hand, activation_hand, hand_interp)
        print('      -- Calculating HAND Membership ... [min thresholds: %6.2f' % (hand_lowerlimit),'; max threshold: %6.2f]' % (hand_upperlimit)) 
        slope_membership = skfuzzy.interp_membership(x_slope, activation_slope, slope)
        print('      -- Calculating Slope Membership ... [min thresholds: 0 deg; max threshold: 15 deg]') 
        area_membership = np.zeros_like(radar_array)
        
        print('      -- Calculating Area Membership ... [This may take a while!]')
        for x in tqdm(range(1, np.max(labeled_image))):
            #clear_output(wait=True)
            #np.putmask(area_membership,labeled_image==x,skfuzzy.interp_membership(x_area, activation_area, np.sum(labeled_image==x)))
            np.putmask(area_membership,labeled_image==x,skfuzzy.interp_membership(x_area, activation_area, label_areas[x-1]))
            
        water_gT=gdal_get_geotransform(waterfile)
        water_proj4=get_proj4(waterfile)
        
        # Uncomment these following lines if you want more intermediary files to be saved off as GeoTIFFs
            #filename, ext = os.path.basename(tiff).split('.')
            #filename, ext = Path(tiff).name.split('.')
            #outfile = f"{mask_directory}/{filename}_dbmembership.{ext}"
            #gdal_write(db_membership, water_gT, filename=outfile, srs_proj4=water_proj4.srs, nodata=np.nan, format=gdal.GDT_Float32)
            #outfile = f"{mask_directory}/{filename}_handmembership.{ext}"
            #gdal_write(hand_membership, water_gT, filename=outfile, srs_proj4=water_proj4.srs, nodata=np.nan, format=gdal.GDT_Float32)
            #outfile = f"{mask_directory}/{filename}_slopemembership.{ext}"
            #gdal_write(slope_membership, water_gT, filename=outfile, srs_proj4=water_proj4.srs, nodata=np.nan, format=gdal.GDT_Float32)
            #outfile = f"{mask_directory}/{filename}_areamembership.{ext}"
            #gdal_write(area_membership, water_gT, filename=outfile, srs_proj4=water_proj4.srs, nodata=np.nan, format=gdal.GDT_Float32)
        
       
        print('   - Combine all fuzzy membership functions to make final flood mapping decision')
        dbmask = db_membership != 0.0
        handmask = hand_membership != 0.0
        slopemask = slope_membership != 0.0
        areamask = area_membership != 0.0
        combinedm = dbmask * handmask * slopemask * areamask
        combinedmask = np.zeros_like(radar_array)
        sel = np.ones_like(dbarray)
        combinedmask[combinedm] = sel[combinedm]
        combinedweights = ((db_membership + hand_membership + slope_membership + area_membership) / 4.0) * combinedmask 
        acceptance = combinedweights > 0.45
        
        # Uncomment these following lines if you want more intermediary files to be saved off as GeoTIFFs
        #outfile = f"{mask_directory}/{filename}_totalmembership.{ext}"
        #gdal_write(combinedweights, water_gT, filename=outfile, srs_proj4=water_proj4.srs, nodata=np.nan, format=gdal.GDT_Float32)
        
        sel = np.ones_like(radar_array)
        floodmap = np.zeros_like(radar_array)
        floodmap[acceptance] = sel[acceptance]
        np.putmask(floodmap,radar_array==0 , 0)
        if 'vv' in tiff or 'VV' in tiff:
            floodmap_vv = floodmap
        else:
            floodmap_vh = floodmap

        # Export flood maps as GeoTIFFs
        #filename, ext = os.path.basename(tiff).split('.')
        #outfile = f"{mask_directory}/{filename}_final_water_mask.{ext}"
        #write_mask_to_file(floodmap, outfile, f.GetProjection(), f.GetGeoTransform())
        del floodmap
        print('   - Processing time: %6.2f minutes' % ((time.time() - start)/60.0))
        print('-----------------------------------------------------------------------------------')
        print(' ')
    
    # Combine VV and VH flood maps to produce a combined flood mapping product
    comb = floodmap_vh + floodmap_vv
    comb_mask = comb > 0
    flood_comb = np.zeros_like(radar_array)
    flood_comb[comb_mask] = sel[comb_mask]
    filename, ext = Path(tiff).name.split('.')
    outfile = f"{mask_directory}/{filename[:-3]}_fcWM.{ext}"
    write_mask_to_file(flood_comb, outfile, f.GetProjection(), f.GetGeoTransform())

# Version Log

<font face="Calibri" size="2" color="gray"> <i> Version 1.01 - Franz J Meyer; 12/15/2020

Recent Changes:
- Post processing was integrated
- Notebook was trimmed down to the necessary parts
- Analysis of results moved into a separate notebook
</i></font>